## Bibliotecas

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import models, transforms
from PIL import Image
import pandas as pd
import numpy as np

## Configurações

In [27]:
dataset_csv = "/home/tatmiki/Documentos/Prog_Projects/cnn_peixes/data/features_dataset.csv"  # pasta principal com subpastas das classes
model_path = "/home/tatmiki/Documentos/Prog_Projects/cnn_peixes/notebooks/trained/v1_10ep.pth"  # caminho do modelo treinado

device = "cuda" if torch.cuda.is_available() else "cpu"

top_k_classes = 3
top_k_similar_images = 3

num_classes = 30

## Transform

In [28]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

## Carrega o modelo ResNet50 para classificação e extração de características

In [29]:
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)

model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

model.load_state_dict(torch.load(model_path, map_location=device))

model.to(device)
model.eval()

feature_extractor = nn.Sequential(*list(model.children())[:-1])
feature_extractor.eval()

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


## Função de classificação e busca por similariadade

In [ ]:
def classify_and_find_similar(img_path, csv_path=dataset_csv):
    # Carrega CSV com features
    df = pd.read_csv(csv_path)
    df['vetor_caracteristicas'] = df['vetor_caracteristicas'].apply(
        lambda x: np.array(list(map(float, x.split())))
    )

    # Carrega imagem de entrada
    img = Image.open(img_path).convert("RGB")
    img_tensor = transform(img).unsqueeze(0).to(device)

    # Classificação Top-3
    with torch.no_grad():
        outputs = model(img_tensor)
        probs = torch.softmax(outputs, dim=1).cpu().numpy().flatten()
    
    # Top-3 classes
    top3_idx = probs.argsort()[-top_k_classes:][::-1]
    top3_classes = []
    for idx in top3_idx:
        # Obter nome da classe a partir do CSV
        class_name = df[df['id_classe'] == idx]['nome_classe'].iloc[0]
        top3_classes.append({
            "id_classe": idx,
            "nome_classe": class_name,
            "probabilidade": float(probs[idx])
        })

    # Extrair vetor de características da imagem
    with torch.no_grad():
        feats = feature_extractor(img_tensor)
        feats = feats.view(feats.size(0), -1).cpu().numpy().flatten()

    # Encontrar imagens mais semelhantes
    similar_images = []
    for cls in top3_classes:
        class_id = cls['id_classe']
        class_imgs = df[df['id_classe'] == class_id].copy()

        # calcular similaridade cosseno
        class_imgs['sim_cosseno'] = class_imgs['vetor_caracteristicas'].apply(
            lambda x: F.cosine_similarity(torch.tensor(feats), torch.tensor(x), dim=0).item()
        )

        # pegar top-3 imagens
        top_imgs = class_imgs.sort_values(by='sim_cosseno', ascending=False).head(top_k_similar_images)
        for _, row in top_imgs.iterrows():
            similar_images.append({
                "id_classe": row['id_classe'],
                "caminho_imagem": row['caminho_imagem'],
                "nome_classe": row['nome_classe'],
                "sim_cosseno": row['sim_cosseno']
            })

    return top3_classes, similar_images

## Teste da busca por similaridade

In [31]:
test_image = "/home/tatmiki/Documentos/Prog_Projects/cnn_peixes/data/test_app/images.jpg"
top3_classes, top9_images = classify_and_find_similar(test_image)

print("=== Top-3 Classes ===")
for cls in top3_classes:
    print(cls)

print("\n=== Top-9 Imagens mais semelhantes ===")
for img_info in top9_images:
    print(img_info)

=== Top-3 Classes ===
{'id_classe': np.int64(0), 'nome_classe': 'Abramis brama', 'probabilidade': 0.9989234805107117}
{'id_classe': np.int64(3), 'nome_classe': 'Aspius aspius', 'probabilidade': 0.0005711693083867431}
{'id_classe': np.int64(5), 'nome_classe': 'Blicca bjoerkna', 'probabilidade': 0.00029558647656813264}

=== Top-9 Imagens mais semelhantes ===
{'id_classe': 0, 'caminho_imagem': '/home/tatmiki/Documentos/Prog_Projects/cnn_peixes/data/dataset/Abramis brama/19606-620mm.jpg', 'nome_classe': 'Abramis brama', 'sim_cosseno': 0.8689644273286656}
{'id_classe': 0, 'caminho_imagem': '/home/tatmiki/Documentos/Prog_Projects/cnn_peixes/data/dataset/Abramis brama/115899-650mm.jpg', 'nome_classe': 'Abramis brama', 'sim_cosseno': 0.8665059749174702}
{'id_classe': 0, 'caminho_imagem': '/home/tatmiki/Documentos/Prog_Projects/cnn_peixes/data/dataset/Abramis brama/126604-500mm.jpg', 'nome_classe': 'Abramis brama', 'sim_cosseno': 0.8534490858581978}
{'id_classe': 3, 'caminho_imagem': '/home/tat